## Taxi Trajectory Prediction

Here are the links to papers and github repo with solution

https://arxiv.org/pdf/1508.00021.pdf  
https://github.com/adbrebs/taxi

In [1]:
! ls /home/ubuntu/taxi

arrival-clusters.pkl			    taxi1.ipynb   train.csv
arrivals.pkl				    test2pickle   val_idx.pkl
__MACOSX				    test.csv	  X_train.pkl
metaData_taxistandsID_name_GPSlocation.csv  tmp		  X_valid.pkl
models					    train2pickle


In [2]:
import torch

In [3]:
from fastai.imports import *
from fastai.structured import *

In [4]:
path = "/home/ubuntu/taxi/"

In [4]:
# import zipfile
# zip_ref = zipfile.ZipFile('train.csv.zip', 'r')
# zip_ref.extractall(path)
# zip_ref.close()

In [5]:
train = pd.read_csv("train.csv")

In [6]:
test = pd.read_csv("test.csv")

In [7]:
train.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [8]:
list(train.POLYLINE[:1])

['[[-8.618643,41.141412],[-8.618499,41.141376],[-8.620326,41.14251],[-8.622153,41.143815],[-8.623953,41.144373],[-8.62668,41.144778],[-8.627373,41.144697],[-8.630226,41.14521],[-8.632746,41.14692],[-8.631738,41.148225],[-8.629938,41.150385],[-8.62911,41.151213],[-8.629128,41.15124],[-8.628786,41.152203],[-8.628687,41.152374],[-8.628759,41.152518],[-8.630838,41.15268],[-8.632323,41.153022],[-8.631144,41.154489],[-8.630829,41.154507],[-8.630829,41.154516],[-8.630829,41.154498],[-8.630838,41.154489]]']

In [9]:
list(test.POLYLINE[:1])

['[[-8.585676,41.148522],[-8.585712,41.148639],[-8.585685,41.148855],[-8.58573,41.148927],[-8.585982,41.148963],[-8.586396,41.148954],[-8.586072,41.14872],[-8.586324,41.147847],[-8.586999,41.14746],[-8.586576,41.147154],[-8.584884,41.146623]]']

In [10]:
test.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,T1,B,NaN,15.0,20000542,1408039037,A,False,"[[-8.585676,41.148522],[-8.585712,41.148639],[..."
1,T2,B,NaN,57.0,20000108,1408038611,A,False,"[[-8.610876,41.14557],[-8.610858,41.145579],[-..."
2,T3,B,NaN,15.0,20000370,1408038568,A,False,"[[-8.585739,41.148558],[-8.58573,41.148828],[-..."
3,T4,B,NaN,53.0,20000492,1408039090,A,False,"[[-8.613963,41.141169],[-8.614125,41.141124],[..."
4,T5,B,NaN,18.0,20000621,1408039177,A,False,"[[-8.619903,41.148036],[-8.619894,41.148036]]"


In [11]:
meta = pd.read_csv("metaData_taxistandsID_name_GPSlocation.csv")

In [12]:
meta.head()

,ID,Descricao,Latitude,Longitude
0,1,Agra,41.1771457135,-8.609670
1,2,Alameda,41.15618964,-8.591064
2,3,Aldoar,41.1705249231,-8.665876
3,4,Alfândega,41.1437639911,-8.621803
4,5,Amial,41.1835097223,-8.612726


In [13]:
meta.shape, test.shape, train.shape

((63, 4), (320, 9), (1710670, 9))

Ok. So we have 1.7 million training rides and only 320 test rides. Metadata is just having name of cities for lat/long. 

In [15]:
train.dtypes

TRIP_ID           int64
CALL_TYPE        object
ORIGIN_CALL     float64
ORIGIN_STAND    float64
TAXI_ID           int64
TIMESTAMP         int64
DAY_TYPE         object
MISSING_DATA       bool
POLYLINE         object
dtype: object

### Step 1. Feature extraction

Winner's process to convert csv to hdf5 and extract features. I would prefer working on `pandas` for the same as it will allow me to look at how data looks. Replicating all steps present in `csv_to_hdf5.py` to make pandas df

In [19]:
train2 = train.copy()
test2 = test.copy()
meta2 = meta.copy()

Below code proves that all the taxi ids in test set are also present in training set. We don't have any new taxi id in test data. But there are **20 origin calls** that are not in training data but are in test data. 

In [24]:
s=0
for l in test.TAXI_ID.unique():
    if l not in train.TAXI_ID.unique():
        s = s+1
p=0
g = []
for l in test.ORIGIN_CALL.unique():
    if l not in train.ORIGIN_CALL.unique():
        p = p+1
        g.append(l)
s,p

(0, 20)

`g` as saved above has list of origin_calls that are in test but not in train

In [ ]:
g[1:]

Pre-calculated below on train set


In [58]:
lat_mean = 41.15731
lat_std = 0.074120656
long_mean = -8.6161413
long_std = 0.057200309

Making a new feature with only first 5 and last 5 coordinates. In cases where we don't have enough coordinates, we do `edge` padding (i.e. just adding first and last values in leftmost and rightmost sides respectively)

In [70]:
def start_stop_inputs(k, train, test):
    
    """
    Function used to take first k and last k coordinates from given polyline.
    For train - if total coordinates < 2 then return none, if > 2*k then return first k and last k
    and if > 2 but < 2*k then do edge padding and return.
    
    """
    result_train = []
    result_test = []
    
    for l in train[['LONGITUDE','LATITUDE']].iterrows():
        if len(l[1][0]) < 2 or len(l[1][1]) < 2:
            result_train.append(np.nan)
        elif len(l[1][0][:-1]) >= 2*k:
            result_train.append(np.concatenate([l[1][0][0:k],l[1][0][-k:],
                                          l[1][1][0:k],l[1][1][-k:]]).flatten())
        else:
            l1 = np.lib.pad(l[1][0][:-1], (0,4*k-len(l[1][0][:-1])), mode='edge')
            l2 = np.lib.pad(l[1][1][:-1], (0,4*k-len(l[1][1][:-1])), mode='edge')
            result_train.append(np.concatenate([l1[0:k],l1[-k:],l2[0:k],l2[-k:]]).flatten())

    for l in test[['LONGITUDE','LATITUDE']].iterrows(): 
        if len(l[1][0]) < 1 or len(l[1][1]) < 1:
            result_test.append(np.nan)
        elif len(l[1][0]) >= 2*k:
            result_test.append(np.concatenate([l[1][0][0:k],l[1][0][-k:],l[1][1][0:k],l[1][1][-k:]]).flatten())
        else:
            l1 = np.lib.pad(l[1][0], (0,4*k-len(l[1][0])), mode='edge')
            l2 = np.lib.pad(l[1][1], (0,4*k-len(l[1][1])), mode='edge')
            result_test.append(np.concatenate([l1[0:k],l1[-k:],l2[0:k],l2[-k:]]).flatten())
    
    return pd.Series(result_train), pd.Series(result_test)

In [75]:
import ast
def feature_ext(train, test):   
    """
    Function used to extract model features.
    • extract lat/long from polyline
    • lat/long normalized using pre-calculated mean and std
    • origin_call and taxi_id converted to continuous numbers to get embeddings later
    • categories convert to cat codes
    
    """
    
    # origin_call -- train
    train.ORIGIN_CALL.fillna(value = -1, inplace=True)
    id_uniq = train.ORIGIN_CALL.unique()
    
    origin_call_dict = {o:i for i,o in enumerate(id_uniq)}

    train['ORIGIN_CALL'] = train.ORIGIN_CALL.apply(lambda x: 0 if x == -1 
                                                   or x == '' else origin_call_dict[x])
    
    
    # origin_call --test (g list of ids in test not in train)
    test.ORIGIN_CALL.fillna(value = -1, inplace=True)

    test['ORIGIN_CALL'] = test.ORIGIN_CALL.apply(lambda x: 0 if x == -1 or x == '' 
                                              else (-2 if x in g[1:] else origin_call_dict[x]))
    
    
    # origin_stand
    train['ORIGIN_STAND']= pd.Series([0 if pd.isnull(x) or x=='' else int(x) for x in train["ORIGIN_STAND"]])
    test['ORIGIN_STAND']= pd.Series([0 if pd.isnull(x) or x=='' else int(x) for x in test["ORIGIN_STAND"]])
    
    # taxi_id
    id_uniq = train.TAXI_ID.unique()
    taxi_id_dict = {o:i for i,o in enumerate(id_uniq)}
    train['TAXI_ID'] = train.TAXI_ID.apply(lambda x: taxi_id_dict[x])
    test['TAXI_ID'] = test.TAXI_ID.apply(lambda x: taxi_id_dict[x])

    # day_type and call_type
    # want 0 for A, 1 for B and 2 for C
    train['DAY_TYPE'] = pd.Series([ord(x[0]) - ord('A') for x in train['DAY_TYPE']])
    train['CALL_TYPE'] = pd.Series([ord(x[0]) - ord('A') for x in train['CALL_TYPE']])
    test['DAY_TYPE'] = pd.Series([ord(x[0]) - ord('A') for x in test['DAY_TYPE']])
    test['CALL_TYPE'] = pd.Series([ord(x[0]) - ord('A') for x in test['CALL_TYPE']])
    
    
    # polyline
    polyline1 = pd.Series([ast.literal_eval(x) for x in train['POLYLINE']])
    polyline2 = pd.Series([ast.literal_eval(x) for x in test['POLYLINE']])
    
    # latitude and longitude
    train['LATITUDE'] = pd.Series([np.array([point[1] for point in poly],dtype=np.float32) for poly in polyline1])
    train['LONGITUDE'] = pd.Series([np.array([point[0] for point in poly],dtype=np.float32) for poly in polyline1])
    
    test['LATITUDE'] = pd.Series([np.array([point[1] for point in poly],dtype=np.float32) for poly in polyline2])
    test['LONGITUDE'] = pd.Series([np.array([point[0] for point in poly],dtype=np.float32) for poly in polyline2])
    
    # target variable i.e. last coordinates
    train['TARGET'] = pd.Series([[l[1][0][-1], l[1][1][-1]] if len(l[1][0]) > 1 
                                else np.nan for l in train[['LONGITUDE','LATITUDE']].iterrows()])

    # normalized lat long. We need normalized data for neural nets and this is like cont. variable (not cat)
    train['LATITUDE'] = pd.Series([(t-lat_mean)/lat_std for t in train['LATITUDE']])
    test['LATITUDE'] = pd.Series([(t-lat_mean)/lat_std for t in test['LATITUDE']])
    
    train['LONGITUDE'] = pd.Series([(t-long_mean)/long_std for t in train['LONGITUDE']])
    test['LONGITUDE'] = pd.Series([(t-long_mean)/long_std for t in test['LONGITUDE']])

    # first 5 and last 5 features
    train['COORD_FEATURES'], test['COORD_FEATURES'] = start_stop_inputs(5, train, test)
    
    # day of week variable
    train['DAY_OF_WEEK'] = pd.Series([datetime.datetime.fromtimestamp(t).weekday() for t in train['TIMESTAMP']])
    test['DAY_OF_WEEK'] = pd.Series([datetime.datetime.fromtimestamp(t).weekday() for t in test['TIMESTAMP']])

    # hour of day
    train['HOUR'] = pd.Series([datetime.datetime.fromtimestamp(t).hour for t in train['TIMESTAMP']])
    test['HOUR'] = pd.Series([datetime.datetime.fromtimestamp(t).hour for t in test['TIMESTAMP']])

    # week of year
    train['WEEK_OF_YEAR'] = pd.Series([datetime.datetime.fromtimestamp(t).isocalendar()[1] for t in train['TIMESTAMP']])
    test['WEEK_OF_YEAR'] = pd.Series([datetime.datetime.fromtimestamp(t).isocalendar()[1] for t in test['TIMESTAMP']])

        
    #data = data.dropna()

    return train, test

In [77]:
train2, test2 = train.copy(), test.copy()

In [ ]:
train2, test2 = feature_ext(train2, test2)

In [88]:
train2.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LATITUDE,LONGITUDE,TARGET,COORD_FEATURES,DAY_OF_WEEK,HOUR,WEEK_OF_YEAR
0,1372636858620000589,2,0,0,0,1372636858,0,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...","[-0.21451, -0.214974, -0.199688, -0.182087, -0...","[-0.0437321, -0.0412145, -0.0731591, -0.105104...","[-8.63084, 41.1545]","[-0.0437321, -0.0412145, -0.0731591, -0.105104...",0,0,27
1,1372637303620000596,1,0,7,1,1372637303,0,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...","[0.0339161, 0.0345337, 0.0378275, 0.0429227, 0...","[-0.414429, -0.423249, -0.455494, -0.494991, -...","[-8.66574, 41.1707]","[-0.414429, -0.423249, -0.455494, -0.494991, -...",0,0,27
2,1372636951620000320,2,0,0,2,1372636951,0,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...","[-0.228715, -0.228818, -0.229796, -0.228561, -...","[0.0555529, 0.048317, 0.0336785, 0.0239251, 0....","[-8.61597, 41.1405]","[0.0555529, 0.048317, 0.0336785, 0.0239251, 0....",0,0,27
3,1372636854620000520,2,0,0,3,1372636854,0,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...","[-0.0723098, -0.0724127, -0.0725671, -0.072206...","[0.724872, 0.724405, 0.724572, 0.725189, 0.724...","[-8.608, 41.1429]","[0.724872, 0.724405, 0.724572, 0.725189, 0.724...",0,0,27
4,1372637091620000337,2,0,0,4,1372637091,0,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...","[0.312708, 0.313068, 0.306789, 0.291092, 0.285...","[-0.5219, -0.521117, -0.522834, -0.536055, -0....","[-8.68727, 41.1781]","[-0.5219, -0.521117, -0.522834, -0.536055, -0....",0,0,27


In [89]:
test2.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LATITUDE,LONGITUDE,COORD_FEATURES,DAY_OF_WEEK,HOUR,WEEK_OF_YEAR
0,T1,1,0,15,150,1408039037,0,False,"[[-8.585676,41.148522],[-8.585712,41.148639],[...","[-0.118578, -0.116982, -0.1141, -0.113122, -0....","[0.532604, 0.531971, 0.532454, 0.531671, 0.527...","[0.532604, 0.531971, 0.532454, 0.531671, 0.527...",3,17,33
1,T2,1,0,57,306,1408038611,0,False,"[[-8.610876,41.14557],[-8.610858,41.145579],[-...","[-0.158413, -0.158258, -0.155736, -0.150024, -...","[0.0920491, 0.0923659, 0.0915823, 0.0996017, 0...","[0.0920491, 0.0923659, 0.0915823, 0.0996017, 0...",3,17,33
2,T3,1,0,15,260,1408038568,0,False,"[[-8.585739,41.148558],[-8.58573,41.148828],[-...","[-0.118063, -0.11446, -0.112505, -0.111887, -0...","[0.531504, 0.531671, 0.531821, 0.5219, 0.52490...","[0.531504, 0.531671, 0.531821, 0.5219, 0.52490...",3,17,33
3,T4,1,0,53,40,1408039090,0,False,"[[-8.613963,41.141169],[-8.614125,41.141124],[...","[-0.217753, -0.21837, -0.221047, -0.222488, -0...","[0.0380801, 0.0352457, 0.0184065, 0.0151053, 0...","[0.0380801, 0.0352457, 0.0184065, 0.0151053, 0...",3,17,33
4,T5,1,0,18,33,1408039177,0,False,"[[-8.619903,41.148036],[-8.619894,41.148036]]","[-0.125114, -0.125114]","[-0.0657565, -0.0656064]","[-0.0657565, -0.0656064, -0.0656064, -0.065606...",3,17,33


Saving transformed files as `pickle`

In [ ]:
import pickle

Saving files

In [101]:
train2.to_pickle(path+'train2pickle')
test2.to_pickle(path+'test2pickle')

Loading train and test. **Can directly run this**

In [5]:
train2 = pd.read_pickle('train2pickle')
test2 = pd.read_pickle('test2pickle')

### Step 2: Meanshift clustering

In [6]:
import os
import pickle
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.datasets.samples_generator import make_blobs
from itertools import cycle
import scipy.misc

In [37]:
# checking how to index and subset lat and long (lat is in 1 and long in 0th)
train2.TARGET[:10][1][1], train2.TARGET[:10][1][0]

(41.17067, -8.66574)

We don't have polyline data for all the training data. For those, we have `nan` in `TARGET` variable and those won't contribute in centroid calculation. 

In [40]:
print("Generating arrival point list")
dests = []

# for l in train2[['LONGITUDE','LATITUDE']].iterrows():
#     if len(l[1][0]) == 0 or len(l[1][1]) == 0: continue
#     dests.append((l[1][1][-1],  l[1][0][-1])) # latitude, longitude

target = train2.TARGET.fillna(0)    
for l in target:
    if l == 0 : continue
    dests.append((l[1],  l[0])) # latitude, longitude
    
pts = np.array(dests)

with open(os.path.join(path, "arrivals.pkl"), "wb") as f:
    pickle.dump(pts, f, protocol=pickle.HIGHEST_PROTOCOL)

print("Doing clustering")
bw = estimate_bandwidth(pts, quantile=.1, n_samples=1000)
print(bw)
bw = 0.001 

ms = MeanShift(bandwidth=bw, bin_seeding=True, min_bin_freq=5)
ms.fit(pts)
cluster_centers = ms.cluster_centers_

print("Clusters shape: ", cluster_centers.shape)

with open(os.path.join(path, "arrival-clusters.pkl"), "wb") as f:
    pickle.dump(cluster_centers, f, protocol=pickle.HIGHEST_PROTOCOL)

Generating arrival point list
Doing clustering
0.0215466177411
Clusters shape:  (3422, 2)


I tried above code for --  
1. scaled coordinates. It doesn't give even close number of clusters. got **22644 clusters**. Not quite right. 
2. unscaled coordinates. Got **3422 clusters**. Winner had 3392 clusters. CLOSE and will keep it

### Step 3: Making Validation Split

As given in winner's solution, we are taking validation set as last year's data with same timestamp as of test data timestamps

In [7]:
# Cuts of the test set minus 1 year
cuts = [
    1376503200, # 2013-08-14 18:00
    1380616200, # 2013-10-01 08:30
    1381167900, # 2013-10-07 17:45
    1383364800, # 2013-11-02 04:00
    1387722600  # 2013-12-22 14:30
]

In [22]:
val_indices = []
index = 0
for index, row in train2.iterrows():
    time = row['TIMESTAMP']
    latitude = row['LATITUDE']
    for ts in cuts:
        if time <= ts and time + 15 * (len(latitude) - 1) >= ts:
            val_indices.append(index)
            break
    index += 1

saving and reloading validation indexes

In [23]:
with open(os.path.join(path, "val_idx.pkl"), "wb") as f:
    pickle.dump(val_indices, f)

In [8]:
with open(os.path.join(path, "val_idx.pkl"), "rb") as f:
    val_indices = pickle.load(f)

validation set

In [53]:
X_valid = train2.iloc[val_indices]

Let's check if we have right timestamps in validation set or not

In [61]:
lt = []
for d in X_valid['TIMESTAMP']:
    lt.append(datetime.date.fromtimestamp(d))
    
set(lt)

{datetime.date(2013, 8, 14),
 datetime.date(2013, 10, 1),
 datetime.date(2013, 10, 7),
 datetime.date(2013, 11, 2),
 datetime.date(2013, 12, 22)}

Yup. above dates match with our dates of test set. Just 1 previous year

In [30]:
X_train = train2.drop(train2.index[[val_indices]])

In [31]:
len(X_valid), len(X_train)

(304, 1710366)

Validation set length is similar as test set length

In [9]:
with open(os.path.join(path, "arrival-clusters.pkl"), "rb") as f:
    cluster_centers2 = pickle.load(f)

In [11]:
len(cluster_centers2)

3422

Saving `X_train` and `X_valid`

In [32]:
X_train.to_pickle(path + 'X_train.pkl')
X_valid.to_pickle(path + 'X_valid.pkl')

Reading validation and training files. Next time can directly read from here

In [10]:
X_train = pd.read_pickle('X_train.pkl')
X_valid = pd.read_pickle('X_valid.pkl')

### Step 4. Defining Loss function

They used `equirectangular distance` as loss function which performed better than original kaggle metric (`Haversine distance`). Actually equirectangular distance is just a simpler approximation to haversine distance

Will keep long in 0 and lat in 1 as there in target column

In [11]:
rearth = 6371
deg2rad = 3.141592653589793 / 180.

def erdist(a, b):
    lat1 = a[:, 1] * deg2rad
    lon1 = a[:, 0] * deg2rad
    lat2 = b[:, 1] * deg2rad
    lon2 = b[:, 0] * deg2rad
    x = (lon2-lon1) * torch.cos((lat1+lat2)/2)
    y = (lat2-lat1)
    return torch.sqrt(x**2 + y**2) * rearth

### Step 5: DL

Hidden layer is basically telling probability of observation being in ith cluster. So it will have as many outputs as number of clusters. (3422). It includes training for `probabilities`  

Now sum of all the `probabilities` should be 1. Therefore we will use `softmax` over a trained linear layer


On these 3422 output from hidden, we take weighted avg which will just be sum of multiplied probability into cluster center (i.e. sum of 3422 elements). It includes no traning, just using previous probabilities and calculating weighted avg.   

Think of this operation as a linear layer with `weights = centers (i.e. fixed) and input = probabilities`



Layers to add:  
1. Pass input data to a **dense layer** which has 500 outputs and those **500 to relu**. 
1. Pass coming data to a **linear layer** which has 3422 outputs
2. These 3422 outputs passed to a **softmax** (which will give p)
3. These 3422 p's will go to **linear layer** which **does not** update weights.  just multiply p and c . 
4. 


#### Analysing cardinality for embeddings

In [12]:
cols = ['ORIGIN_CALL', 'TAXI_ID', 'ORIGIN_STAND', 'HOUR', 'DAY_OF_WEEK', 'WEEK_OF_YEAR']

In [13]:
for i in cols:
    print(i, '--', 'Possible values', len(train2[i].unique()) ,'||', 'Embedding size', 10)

ORIGIN_CALL -- Possible values 57106 || Embedding size 10
TAXI_ID -- Possible values 448 || Embedding size 10
ORIGIN_STAND -- Possible values 64 || Embedding size 10
HOUR -- Possible values 24 || Embedding size 10
DAY_OF_WEEK -- Possible values 7 || Embedding size 10
WEEK_OF_YEAR -- Possible values 52 || Embedding size 10


They took embedding of size 10 for all the above features

In [14]:
emb_szs = []
for i in cols:
    emb_szs.append((len(train2[i].unique()), 10))
emb_szs

[(57106, 10), (448, 10), (64, 10), (24, 10), (7, 10), (52, 10)]

In [15]:
cat_vars = ['ORIGIN_CALL', 'TAXI_ID', 'ORIGIN_STAND', 'HOUR', 'DAY_OF_WEEK', 'WEEK_OF_YEAR']
#cont_vars = 

#### Getting continuous features

In [47]:
X_train[:1]

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LATITUDE,LONGITUDE,TARGET,COORD_FEATURES,DAY_OF_WEEK,HOUR,WEEK_OF_YEAR
0,1372636858620000589,2,0,0,0,1372636858,0,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...","[-0.21451, -0.214974, -0.199688, -0.182087, -0...","[-0.0437321, -0.0412145, -0.0731591, -0.105104...","[-8.63084, 41.1545]","[-0.0437321, -0.0412145, -0.0731591, -0.105104...",0,0,27


In [16]:
# replacing NA with 0s
# k = 5

X_train.COORD_FEATURES.fillna(0, inplace = True)
X_valid.COORD_FEATURES.fillna(0, inplace = True)

In [17]:
def get_features(df):
    
    cols = ['ORIGIN_CALL', 'TAXI_ID', 'ORIGIN_STAND', 'HOUR', 'DAY_OF_WEEK', 'WEEK_OF_YEAR','MISSING_DATA']
    
    df_feat = df[cols]
    for i in range(10):
        df_feat[f'long_{i}'] = df.COORD_FEATURES.apply(lambda x: 0 if type(x) == int else x[i]) # creating 10 cols for long
        df_feat[f'lat_{i}'] = df.COORD_FEATURES.apply(lambda x: 0 if type(x) == int else x[i+10]) # 10 cols for lat

    df_target = df['TARGET']
               
    return df_feat, df_target

Training and validation sets

In [18]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
train2.COORD_FEATURES.fillna(0, inplace = True)
df, y = get_features(train2)

In [20]:
train_feat, train_target = get_features(X_train)

In [21]:
valid_feat, valid_target = get_features(X_valid) 

#### Defining nn for model

Centers of cluster lats and longs. Needed for last nn layer

In [22]:
cluster_lats = []

for i in cluster_centers2:
    cluster_lats.append(i[0])
    
cluster_longs = []

for i in cluster_centers2:
    cluster_longs.append(i[1])

In [23]:
cluster_lats = np.array(cluster_lats)[:,None]
cluster_longs = np.array(cluster_longs)[:,None]

Mixed input nn model

#### Defining nn

In [24]:
from fastai.structured import *
from fastai.column_data import *
import torch.nn as nn

In [25]:
df[:2]

,ORIGIN_CALL,TAXI_ID,ORIGIN_STAND,HOUR,DAY_OF_WEEK,WEEK_OF_YEAR,MISSING_DATA,long_0,lat_0,long_1,...,long_5,lat_5,long_6,lat_6,long_7,lat_7,long_8,lat_8,long_9,lat_9
0,0,0,0,0,0,27,False,-0.043732,-0.214510,-0.041215,...,-0.262276,-0.038085,-0.256774,-0.037828,-0.256774,-0.037725,-0.256774,-0.037930,-0.256940,-0.038085
1,0,1,7,0,0,27,False,-0.414429,0.033916,-0.423249,...,-0.934763,0.143693,-0.908337,0.157538,-0.882844,0.171279,-0.867572,0.179771,-0.867105,0.180234


In [26]:
def emb_init(x):
    x = x.weight.data
    sc = 2/(x.size(1)+1)
    x.uniform_(-sc,sc)

In [27]:
# they didn't do batchnormm in paper            
class Net(nn.Module):
    def __init__(self, emb_szs, n_cont, emb_drop, out_sz, szs, drops,
                 y_range=None, use_bn=False):
        super().__init__()
        self.embs = nn.ModuleList([nn.Embedding(c, s) for c,s in emb_szs])
        for emb in self.embs: emb_init(emb)
        n_emb = sum(e.embedding_dim for e in self.embs)

        szs = [n_emb+n_cont] + szs        
        self.lin1 = nn.Linear(len(szs), 500)

        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(500, 3422)
        self.soft = nn.Softmax() # got probabilities now
         
        for o in [self.lin1, self.lin2] : kaiming_normal(o.weight.data)
            
        self.bn = nn.BatchNorm1d(n_cont)

    def forward(self, x_cat, x_cont):
        x = [e(x_cat[:,i]) for i,e in enumerate(self.embs)]
        x = torch.cat(x, 1)
        x2 = self.bn(x_cont)
        x = torch.cat([x, x2], 1)
        
        out = self.lin1(x)
        out = self.relu(out)
        out = self.lin2(out)
        out = self.soft()
        outlat = out*cluster_lats
        outlong = out*cluster_longs
            
        return [outlong, outlat]

#### Data loader

Customizing `fast.ai` functions

In [28]:
from fastai.dataset import *
from fastai.learner import *

In [29]:
class ColumnarDatasetCustom(Dataset):
    def __init__(self, cats, conts, y):
        n = len(cats[0]) if cats else len(conts[0])
        self.cats = np.stack(cats, 1).astype(np.int64) if cats else np.zeros((n,1))
        self.conts = np.stack(conts, 1).astype(np.float32) if conts else np.zeros((n,1))
        self.y = np.zeros((n,1)) if y is None else y

    def __len__(self): return len(self.y)

    def __getitem__(self, idx):
        return [self.cats[idx], self.conts[idx], self.y[idx]]

    @classmethod
    def from_data_frames(cls, df_cat, df_cont, y=None):
        cat_cols = [c.values for n,c in df_cat.items()]
        cont_cols = [c.values for n,c in df_cont.items()]
        return cls(cat_cols, cont_cols, y)

    @classmethod
    def from_data_frame(cls, df, cat_flds, y=None):
        return cls.from_data_frames(df[cat_flds], df.drop(cat_flds, axis=1), y)

In [30]:
class ColumnarModelDataCustom(ModelData):
    def __init__(self, path, trn_ds, val_ds, bs, test_ds=None):
        test_dl = DataLoader(test_ds, bs, shuffle=False, num_workers=1) if test_ds is not None else None
        super().__init__(path, DataLoader(trn_ds, bs, shuffle=True, num_workers=1),
            DataLoader(val_ds, bs*2, shuffle=False, num_workers=1), test_dl)

    @classmethod
    def from_data_frames(cls, path, trn_df, val_df, trn_y, val_y, cat_flds, bs, test_df=None):
        test_ds = ColumnarDataset.from_data_frame(test_df, cat_flds) if test_df is not None else None
        return cls(path, ColumnarDatasetCustom.from_data_frame(trn_df, cat_flds, trn_y),
                    ColumnarDatasetCustom.from_data_frame(val_df, cat_flds, val_y), bs, test_ds=test_ds)

    @classmethod
    def from_data_frame(cls, path, val_idxs, df, y, cat_flds, bs, test_df=None):
        ((val_df, trn_df), (val_y, trn_y)) = split_by_idx(val_idxs, df, y)
        return cls.from_data_frames(path, trn_df, val_df, trn_y, val_y, cat_flds, bs, test_df=test_df)
    
    def get_learner_custom(self, emb_szs, n_cont, emb_drop, out_sz, szs, drops,
                y_range=None, use_bn=False, **kwargs):
        model = Net(emb_szs, n_cont, emb_drop, out_sz, szs, drops, y_range, use_bn)
        return StructuredLearnerCustom(self, StructuredModelCustom(to_gpu(model)), opt_fn=optim.SGD, **kwargs)

In [65]:
class StructuredLearnerCustom(Learner):
    def __init__(self, data, models, **kwargs):
        super().__init__(data, models, **kwargs)
        #self.crit = F.mse_loss
        self.crit = erdist

class StructuredModelCustom(BasicModel):
    def get_layer_groups(self):
        m=self.model
        return [m.embs, children(m.lin1),m.lin2]

fitting model using above defined functions

In [66]:
md = ColumnarModelDataCustom.from_data_frame(path, val_indices, df, y, cat_flds=cat_vars, bs=200)

In [68]:
m = md.get_learner_custom(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y)

In [69]:
m

Net (
  (embs): ModuleList (
    (0): Embedding(57106, 10)
    (1): Embedding(448, 10)
    (2): Embedding(64, 10)
    (3): Embedding(24, 10)
    (4): Embedding(7, 10)
    (5): Embedding(52, 10)
  )
  (lin1): Linear (3 -> 500)
  (relu): ReLU ()
  (lin2): Linear (500 -> 3422)
  (soft): Softmax ()
  (bn): BatchNorm1d(21, eps=1e-05, momentum=0.1, affine=True)
)

In [70]:
lr = 0.01

In [67]:
num_epochs = 5

In [72]:
m.fit(lr, 3, metrics=[erdist])

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))


  0%|          | 0/8552 [00:00<?, ?it/s]


TypeError: zip argument #71 must support iteration

#### Final Modeling

In [ ]:
class taxiDataset(Dataset):
    def __init__(self, cats, conts, y):
        n = len(cats[0]) if cats else len(conts[0])
        self.cats = np.stack(cats, 1).astype(np.int64) if cats else np.zeros((n,1))
        self.conts = np.stack(conts, 1).astype(np.float32) if conts else np.zeros((n,1))
        self.y = np.zeros((n,1)) if y is None else y

    def __len__(self): return len(self.y)

    def __getitem__(self, idx):
        return [self.cats[idx], self.conts[idx], self.y[idx]]

    @classmethod
    def from_data_frames(cls, df_cat, df_cont, y=None):
        cat_cols = [c.values for n,c in df_cat.items()]
        cont_cols = [c.values for n,c in df_cont.items()]
        return cls(cat_cols, cont_cols, y)

    @classmethod
    def from_data_frame(cls, df, cat_flds, y=None):
        return cls.from_data_frames(df[cat_flds], df.drop(cat_flds, axis=1), y)

In [50]:
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=b)  

In [40]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(md):  
        # Convert torch tensor to Variable
        images = Variable(images.view(-1, 28*28)).cuda()
        labels = Variable(labels).cuda()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

TypeError: 'ColumnarModelDataCustom' object is not iterable

In [ ]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

In [84]:
thnn.NLLLoss??

Object `thnn.NLLLoss` not found.


In [ ]:
def er_dist_f(input, target, weight=None, size_average=True, ignore_index=-100):
    return _functions.thnn.erdist.apply(input, target, weight, size_average, ignore_index)

In [74]:
class erdist(_WeightedLoss):
  

    def __init__(self, weight=None, size_average=True, ignore_index=-100):
        super(erdist, self).__init__(weight, size_average)
        self.ignore_index = ignore_index

    def forward(self, input, target):
        _assert_no_grad(target)
        return F.er_dist_f(input, target, self.weight, self.size_average,
                          self.ignore_index)

In [ ]:
net2 = m
loss=nn.NLLLoss()
learning_rate = 1e-2
optimizer=optim.SGD(net2.parameters(), lr=learning_rate)

for epoch in range(1):
    losses=[]
    dl = iter(md.trn_dl)
    for t in range(len(dl)):
        # Forward pass: compute predicted y and loss by passing x to the model.
        xt, yt = next(dl)
        y_pred = net2(V(xt))
        l = loss(y_pred, V(yt))
        losses.append(l)

        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable weights of the model)
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model parameters
        l.backward()

        # Calling the step function on an Optimizer makes an update to its parameters
        optimizer.step()
    
    val_dl = iter(md.val_dl)
    val_scores = [score(*next(val_dl)) for i in range(len(val_dl))]
    print(np.mean(val_scores))